In [60]:
import json
import glob
import re
import pandas as pd
import os
import statistics

In [2]:
root="/home/jovyan/work"
analysis_results_path = root + "/analysis/results/"

In [3]:
algs = ['OPENSZZ', 'SZZ_UNLEASHED', 'PYSZZ_ag', 'PYSZZ_l', 'PYSZZ_r', 'PYSZZ_ma', 'PYSZZ_ra']

In [4]:
projects = [
    "JacksonXml", "Time", "Collections", "Compress", "Csv", "JacksonCore", "JacksonDatabind", "Gson", "Jsoup",
    "Lang", "Math", "Closure", "Mockito"
]
regressions = []
for project in projects:
    # FOR EACH BUG
    for bug_path in glob.glob("{root}/results/{project}/Bug_*/".format(root=root, project=project)):
        bug_id = re.search(r"Bug_(\d+)", bug_path).group(1)
        result_path = root+"/analysis/results/{project}/Bug_{bug_id}/bug_result.json".format(project=project, bug_id=bug_id)
        # analysis/results/JacksonXml/Bug_1/bug_result.json
        with open(result_path) as f:
            result = json.load(f)
            if result['category'] == "A regression is detected" and result['sub_category'] == "Unique candidates": 
                result['BIC'] = result['BIC_candidates'][0][1]
                regressions.append(result)

In [51]:
for regression in regressions:
    # print(regression['project'], regression['bug'])
    
    # SZZ UNLEASHED
    szz_unleashed_result_path = root+"/results/szz/SZZUnleashed/{id}_SZZUnleashed/results/fix_and_introducers_pairs.json".format(id=regression['id'])
    if os.path.isfile(szz_unleashed_result_path):
        with open(szz_unleashed_result_path) as f:
            szz_unleashed_result = json.load(f)
            szz_unleashed_result = list(map(lambda c: c[1],szz_unleashed_result))
            regression['SZZ_UNLEASHED_LEN_CANDIDATES'] = len(szz_unleashed_result) 
            regression['SZZ_UNLEASHED'] = regression['BIC'] in szz_unleashed_result 
    
    # OPEN SZZ
    
    openszz_result_path = root+"/results/szz/OpenSZZ/{id}_OpenSZZ/suspects.json".format(id=regression['id'])
    if os.path.isfile(openszz_result_path):
        with open(openszz_result_path) as f:
            openszz_result = json.load(f)
            regression['OPENSZZ_LEN_CANDIDATES'] = len(openszz_result) 
            regression['OPENSZZ'] = regression['BIC'] in openszz_result 
            
    
    # PYSZZ
    for alg in ['ag', 'l', 'r', 'ma', 'ra']:
        pyszz_result_path = root+"/results/szz/PySZZ_{alg}/{id}_PySZZ_{alg}/result.json".format(id=regression['id'],alg=alg)
        if os.path.isfile(pyszz_result_path):
            with open(pyszz_result_path) as f:
                pyszz_result = json.load(f)[0]['inducing_commit_hash']
                regression['PYSZZ_{alg}_LEN_CANDIDATES'.format(alg=alg)] = len(pyszz_result) 
                regression['PYSZZ_{alg}'.format(alg=alg)] = regression['BIC'] in pyszz_result

In [52]:
szz_results = pd.DataFrame(regressions)#[['id'] + algs]
szz_results.T

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
id,Time_Bug_1,Compress_Bug_28,Compress_Bug_45,JacksonCore_Bug_11,JacksonCore_Bug_10,JacksonCore_Bug_21,JacksonDatabind_Bug_24,JacksonDatabind_Bug_35,JacksonDatabind_Bug_41,JacksonDatabind_Bug_87,...,Closure_Bug_172,Closure_Bug_175,Closure_Bug_120,Closure_Bug_115,Closure_Bug_21,Closure_Bug_62,Mockito_Bug_1,Mockito_Bug_3,Mockito_Bug_10,Mockito_Bug_7
bug,Bug_1,Bug_28,Bug_45,Bug_11,Bug_10,Bug_21,Bug_24,Bug_35,Bug_41,Bug_87,...,Bug_172,Bug_175,Bug_120,Bug_115,Bug_21,Bug_62,Bug_1,Bug_3,Bug_10,Bug_7
project,Time,Compress,Compress,JacksonCore,JacksonCore,JacksonCore,JacksonDatabind,JacksonDatabind,JacksonDatabind,JacksonDatabind,...,Closure,Closure,Closure,Closure,Closure,Closure,Mockito,Mockito,Mockito,Mockito
fix_pass,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
prev_fails,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
category,A regression is detected,A regression is detected,A regression is detected,A regression is detected,A regression is detected,A regression is detected,A regression is detected,A regression is detected,A regression is detected,A regression is detected,...,A regression is detected,A regression is detected,A regression is detected,A regression is detected,A regression is detected,A regression is detected,A regression is detected,A regression is detected,A regression is detected,A regression is detected
sub_category,Unique candidates,Unique candidates,Unique candidates,Unique candidates,Unique candidates,Unique candidates,Unique candidates,Unique candidates,Unique candidates,Unique candidates,...,Unique candidates,Unique candidates,Unique candidates,Unique candidates,Unique candidates,Unique candidates,Unique candidates,Unique candidates,Unique candidates,Unique candidates
test_name,org.joda.time.TestPartial_Constructors#testCon...,org.apache.commons.compress.archivers.tar.TarA...,org.apache.commons.compress.archivers.tar.TarU...,com.fasterxml.jackson.core.sym.SymbolsViaParse...,com.fasterxml.jackson.core.sym.TestByteBasedSy...,com.fasterxml.jackson.core.filter.TokenVerifyi...,com.fasterxml.jackson.databind.ser.TestConfig#...,com.fasterxml.jackson.databind.jsontype.Wrappe...,com.fasterxml.jackson.databind.interop.Depreca...,com.fasterxml.jackson.databind.deser.TestDateD...,...,testIssue1024,testIssue1101a,testExternalIssue1053,testBug4944818,testUselessCode,testFormatErrorSpaceEndOfLine1,should_capture_arguments_when_args_count_does_...,should_capture_varargs_as_vararg,should_not_raise_a_mockito_exception_about_ser...,discoverDeepMockingOfGenerics
bug_report,https://github.com/JodaOrg/joda-time/issues/93,https://issues.apache.org/jira/browse/COMPRESS...,https://issues.apache.org/jira/browse/COMPRESS...,https://github.com/FasterXML/jackson-core/issu...,https://github.com/FasterXML/jackson-core/issu...,https://github.com/FasterXML/jackson-core/issu...,https://github.com/FasterXML/jackson-databind/...,https://github.com/FasterXML/jackson-databind/...,https://github.com/FasterXML/jackson-databind/...,https://github.com/FasterXML/jackson-databind/...,...,https://storage.googleapis.com/google-code-arc...,https://storage.googleapis.com/google-code-arc...,https://storage.googleapis.com/google-code-arc...,https://storage.googleapis.com/google-code-arc...,https://storage.googleapis.com/google-code-arc...,https://storage.googleapis.com/google-code-arc...,https://github.com/mockito/mockito/issues/188,https://github.com/mockito/mockito/issues/188,https://github.com/mockito/mockito/issues/99,https://github.com/mockito/mockito/issues/128
fix_commit,9a62b06be5d0df8e833ff8583398cca386608cac,ddb5fd6d8e4d5fa9f7fa659c93fb6eb04f238689,02735ad7e6313515a846bc15bcc89cf254f8161f,be4386724ff18232aff492cb9145288df86ea61c,c2823b4de28120d86aeba9215f0231d990a8eb47,96faf2ecd985dfc838e2bf6f6ae4d9d4b310861b,b4ef0a65150a7311ebb9119b1ac6898e33dc0baf,cecd409aa97693beb87c4e9ca34ea7f

In [69]:
n_regressions = len(szz_results.index)
szz_results_table = []
for alg in algs: 
    total = szz_results[alg].sum()
    candidates = szz_results[alg+'_LEN_CANDIDATES'].sum()
    szz_results_table.append({
        'Algorithm': alg,
        'Total': total,
        'Hit rate':  (total * 100 / n_regressions),
        'Avg. # of candidates': candidates / n_regressions
    })
szz_results_table_df = pd.DataFrame(szz_results_table)
szz_results_table.append({
    'Algorithm':'Average',
    'Total': statistics.fmean(szz_results_table_df['Total']),
    'Hit rate': statistics.fmean(szz_results_table_df['Hit rate']),
    'Avg. # of candidates': statistics.fmean(szz_results_table_df['Avg. # of candidates'])
})
pd.DataFrame(szz_results_table).round(decimals=2)

,Algorithm,Total,Hit rate,Avg. # of candidates
0,OPENSZZ,12.00,24.49,0.94
1,SZZ_UNLEASHED,3.00,6.12,12.73
2,PYSZZ_ag,26.00,53.06,1.33
3,PYSZZ_l,9.00,18.37,0.65
4,PYSZZ_r,14.00,28.57,0.65
5,PYSZZ_ma,32.00,65.31,2.00
6,PYSZZ_ra,25.00,51.02,1.41
7,Average,17.29,35.28,2.82


In [39]:
szz_results_only_bool = szz_results[['id'] + algs]
at_least_one_located_regression = szz_results_only_bool[szz_results_only_bool.select_dtypes([bool]).any(1)]
non_located_regressions = szz_results_only_bool.drop(at_least_one_located_regression.index)
non_located_regressions

,id,OPENSZZ,SZZ_UNLEASHED,PYSZZ_ag,PYSZZ_l,PYSZZ_r,PYSZZ_ma,PYSZZ_ra
2,Compress_Bug_45,False,False,False,False,False,False,False
3,JacksonCore_Bug_11,False,False,False,False,False,False,False
4,JacksonCore_Bug_10,False,False,False,False,False,False,False
7,JacksonDatabind_Bug_35,False,False,False,False,False,False,False
10,JacksonDatabind_Bug_59,False,False,False,False,False,False,False
12,JacksonDatabind_Bug_72,False,False,False,False,False,False,False
16,Jsoup_Bug_43,False,False,False,False,False,False,False
19,Jsoup_Bug_15,False,False,False,False,False,False,False
21,Jsoup_Bug_72,False,False,False,False,False,False,False
31,Math_Bug_87,False,False,False,False,False,False,False
